In [1]:
#from mmdet.apis import init_detector, inference_detector, show_result_pyplot
#import mmcv

import matplotlib.pyplot as plt
from glob import glob
from cv2 import cv2
import numpy as np

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import base64
import time
import math
import datetime
import os
from PIL import Image
from io import BytesIO
from tqdm import tqdm
from glob import glob

import torch
import torchvision
import torch.distributed as dist
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from collections import defaultdict, deque

def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

In [15]:
a = json.load(open('data/dacon/test_annotations.json'))

name_all = []
for i in range(len(a['images'])):
    name_all += [a['images'][i]['file_name'].split('.')[0] + '.json']

In [2]:
with open('result_swin_epoch70_ttaflip.bbox_PB71.56.json', 'r') as temp:
    output = json.load(temp)

In [3]:
with open('result_detectors_finetuning_epoch28_tta.bbox.json', 'r') as temp:
    output2 = json.load(temp)

In [175]:
threshold = 0.35

results = {
    'file_name':[], 'class_id':[], 'confidence':[], 'point1_x':[], 'point1_y':[],
    'point2_x':[], 'point2_y':[], 'point3_x':[], 'point3_y':[], 'point4_x':[], 'point4_y':[]
}


for num, out in enumerate(tqdm(output2)):
    
    coord = out['bbox']
    
    x_center = coord[0]
    y_center = coord[1]
    width = coord[2]
    height = coord[3]
    
    x_min = x_center
    y_min = y_center
    x_max = x_center + width
    y_max = y_center + height

    confidence = out['score']

    image_id = 200000 + out['image_id']
    
    if confidence > threshold:

        results['file_name'].append('test_{}'.format(image_id) + '.json')
        results['class_id'].append(out['category_id'] + 1)
        results['confidence'].append(confidence)
        results['point1_x'].append(x_min)
        results['point1_y'].append(y_min)
        results['point2_x'].append(x_max)
        results['point2_y'].append(y_min)
        results['point3_x'].append(x_max)
        results['point3_y'].append(y_max)
        results['point4_x'].append(x_min)
        results['point4_y'].append(y_max)  

100%|███████████████████████████████| 279222/279222 [00:00<00:00, 451363.12it/s]


In [176]:
submission = pd.DataFrame(results)
submission.shape

(25001, 11)

In [292]:
submission.to_csv('sample_submission.csv', index=False)

In [174]:
submission_swin = submission

In [177]:
submission_det = submission

In [178]:
sub1 = submission_swin
sub2 = submission_det

# Ensemble (처음결과(class 안겹치면 추가) 1%정도 향상되었음)

In [14]:
def IoU(sub1, sub2, name):
    
    output_swin = sub1[sub1.file_name.values == name]
    output_det = sub2[sub2.file_name.values == name]   
    
    box1 = (output_swin.point1_x[0], output_swin.point1_y[0], output_swin.point3_x[0], output_swin.point3_y[0])
    box2 = (output_det.point1_x[0], output_det.point1_y[0], output_det.point3_x[0], output_det.point3_y[0])
    
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

In [370]:
results_ensemble = {
    'file_name':[], 'class_id':[], 'confidence':[], 'point1_x':[], 'point1_y':[],
    'point2_x':[], 'point2_y':[], 'point3_x':[], 'point3_y':[], 'point4_x':[], 'point4_y':[]
}

for i in range(len(sub1)):


    
    file_name = sub1['file_name'].values[i]
    class_id = sub1['class_id'].values[i]

    out1 = sub1.iloc[i].values
    out2 = sub2[sub2['file_name'].values == file_name]
    
    results_ensemble['file_name'].append(out1[0])
    results_ensemble['class_id'].append(out1[1])
    results_ensemble['confidence'].append(out1[2])
    results_ensemble['point1_x'].append(out1[3])
    results_ensemble['point1_y'].append(out1[4])
    results_ensemble['point2_x'].append(out1[5])
    results_ensemble['point2_y'].append(out1[6])
    results_ensemble['point3_x'].append(out1[7])
    results_ensemble['point3_y'].append(out1[8])
    results_ensemble['point4_x'].append(out1[9])
    results_ensemble['point4_y'].append(out1[10]) 
    
    output = np.zeros(shape=(len(out2)))
    for j in range(len(out2)):

        out2_temp = out2.values[j] 

        if out2_temp[1] != class_id:
            results_ensemble['file_name'].append(out2_temp[0])
            results_ensemble['class_id'].append(out2_temp[1])
            results_ensemble['confidence'].append(out2_temp[2])
            results_ensemble['point1_x'].append(out2_temp[3])
            results_ensemble['point1_y'].append(out2_temp[4])
            results_ensemble['point2_x'].append(out2_temp[5])
            results_ensemble['point2_y'].append(out2_temp[6])
            results_ensemble['point3_x'].append(out2_temp[7])
            results_ensemble['point3_y'].append(out2_temp[8])
            results_ensemble['point4_x'].append(out2_temp[9])
            results_ensemble['point4_y'].append(out2_temp[10])        

In [371]:
final = pd.DataFrame(results_ensemble)

In [387]:
final.to_csv('sample_submission.csv', index=False)

# WBF

In [ ]:
from ensemble_boxes import *

In [179]:
from ensemble_boxes import *

results = {
    'file_name':[], 'class_id':[], 'confidence':[], 'point1_x':[], 'point1_y':[],
    'point2_x':[], 'point2_y':[], 'point3_x':[], 'point3_y':[], 'point4_x':[], 'point4_y':[]
}

for i in range(len(name_all)):
    
    boxes_list = []
    scores_list = []
    labels_list = []
    iou_thr = []
    
    sub1_info = sub1.iloc[sub1.file_name.values == name_all[i]]
    sub2_info = sub2.iloc[sub2.file_name.values == name_all[i]]
    
    sub1_boxes = sub1_info.iloc[:, [3, 4, 7, 8]].values.tolist()
    sub2_boxes = sub2_info.iloc[:, [3, 4, 7, 8]].values.tolist()
    
    
    sub1_boxes_temp = []
    for a in range(len(sub1_boxes)):
        temp = sub1_boxes[a]
        sub1_boxes_temp.append([i/576 for i in temp])    
    
    sub2_boxes_temp = []
    for b in range(len(sub2_boxes)):
        temp = sub2_boxes[b]
        sub2_boxes_temp.append([i/576 for i in temp])     
    
    
    sub1_scores = sub1_info.iloc[:, 2].values.tolist()
    sub2_scores = sub2_info.iloc[:, 2].values.tolist()   
    
    sub1_labels = sub1_info.iloc[:, 1].values.tolist()
    sub2_labels = sub2_info.iloc[:, 1].values.tolist()     
    
    
    boxes_list.append(sub1_boxes_temp)
    boxes_list.append(sub2_boxes_temp)
    
    scores_list.append(sub1_scores)
    scores_list.append(sub2_scores)
    
    labels_list.append(sub1_labels)
    labels_list.append(sub2_labels)    
    
    boxes, scores, labels = \
    weighted_boxes_fusion(boxes_list, scores_list, labels_list)
    
    
    for k in range(len(boxes)):
        
        x_min = boxes[k][0] * 576
        y_min = boxes[k][1] * 576
        x_max = boxes[k][2] * 576
        y_max = boxes[k][3] * 576
        
    
        results['file_name'].append(name_all[i])
        results['class_id'].append(int(labels[k]))
        results['confidence'].append(scores[k])
        results['point1_x'].append(x_min)
        results['point1_y'].append(y_min)
        results['point2_x'].append(x_max)
        results['point2_y'].append(y_min)
        results['point3_x'].append(x_max)
        results['point3_y'].append(y_max)
        results['point4_x'].append(x_min)
        results['point4_y'].append(y_max)     

In [180]:
submission = pd.DataFrame(results)
submission.shape

(29182, 11)

In [181]:
submission

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,4,0.544002,376.195358,345.577629,402.442383,345.577629,402.442383,375.136585,376.195358,375.136585
1,test_200001.json,1,0.727125,411.598251,201.986715,521.180454,201.986715,521.180454,313.000729,411.598251,313.000729
2,test_200001.json,1,0.709320,116.537038,328.614841,265.288513,328.614841,265.288513,471.040981,116.537038,471.040981
3,test_200002.json,2,0.937803,220.528547,346.668056,343.158062,346.668056,343.158062,484.752605,220.528547,484.752605
4,test_200003.json,2,0.978205,135.345881,13.774350,306.674320,13.774350,306.674320,191.463598,135.345881,191.463598
...,...,...,...,...,...,...,...,...,...,...,...
29177,test_220871.json,4,0.979544,367.296810,158.212910,433.560539,158.212910,433.560539,222.352707,367.296810,222.352707
29178,test_220872.json,1,0.988181,364.491314,377.974388,442.442402,377.974388,442.442402,475.480865,364.491314,475.480865
29179,test_220873.json,1,0.456512,296.541992,432.744873,459.572205,432.744873,459.572205,538.894958,296.541992,538.894958
29180,test_220873.json,1,0.366084,239.410370,172.003784,331.375366,172.003784,331.375366,253.893860,239.410370,253.893860


In [182]:
submission.to_csv('sample_submission.csv', index=False)

## Swin 결과

In [183]:
sub1

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,4,0.470789,375.977661,345.196106,402.688171,345.196106,402.688171,375.625793,375.977661,375.625793
1,test_200001.json,1,0.872736,117.836884,328.171387,265.592194,328.171387,265.592194,469.491577,117.836884,469.491577
2,test_200001.json,1,0.628555,410.153442,199.907333,520.273560,199.907333,520.273560,311.438141,410.153442,311.438141
3,test_200002.json,2,0.992942,220.499466,347.606812,347.888611,347.606812,347.888611,494.842834,220.499466,494.842834
4,test_200003.json,2,0.986874,134.546646,14.677361,299.757629,14.677361,299.757629,190.274445,134.546646,190.274445
...,...,...,...,...,...,...,...,...,...,...,...
25988,test_220870.json,3,0.993573,183.971405,11.252531,212.478729,11.252531,212.478729,94.754250,183.971405,94.754250
25989,test_220871.json,4,0.986197,369.340637,155.639832,432.745850,155.639832,432.745850,218.476562,369.340637,218.476562
25990,test_220872.json,1,0.994161,369.176025,377.532440,440.337891,377.532440,440.337891,474.218231,369.176025,474.218231
25991,test_220873.json,1,0.732168,239.410370,172.003784,331.375366,172.003784,331.375366,253.893860,239.410370,253.893860


## Detector RS 결과

In [184]:
sub2

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,4,0.617215,376.361450,345.868652,402.254883,345.868652,402.254883,374.763428,376.361450,374.763428
1,test_200001.json,1,0.825696,412.698120,203.569611,521.870789,203.569611,521.870789,314.190247,412.698120,314.190247
2,test_200001.json,1,0.545903,114.458954,329.323853,264.803040,329.323853,264.803040,473.517944,114.458954,473.517944
3,test_200002.json,2,0.882663,220.561234,345.612000,337.836487,345.612000,337.836487,473.401672,220.561234,473.401672
4,test_200003.json,2,0.969535,136.159424,12.855190,313.714722,12.855190,313.714722,192.674011,136.159424,192.674011
...,...,...,...,...,...,...,...,...,...,...,...
24996,test_220869.json,4,0.986835,88.387253,237.519379,125.721306,237.519379,125.721306,278.840332,88.387253,278.840332
24997,test_220870.json,3,0.993011,183.762360,11.935522,212.317902,11.935522,212.317902,94.553131,183.762360,94.553131
24998,test_220871.json,4,0.972890,365.225006,160.821198,434.386414,160.821198,434.386414,226.281860,365.225006,226.281860
24999,test_220872.json,1,0.982201,359.749573,378.421692,444.572571,378.421692,444.572571,476.758911,359.749573,476.758911
